In [1]:
import asyncio
import logging
import time
from pathlib import Path

# Pyrit imports
from pyrit.common import IN_MEMORY, initialize_pyrit
from pyrit.prompt_target import OpenAIChatTarget, AHGPTHttpTarget
from pyrit.score.evaluator import Evaluator
from pyrit.orchestrator.multi_turn.ah_gpt.ah_gpt_orchestrator import AHGPTOrchestrator, RTOSystemPromptPaths
from pyrit.common.text_helper import save_html_report, generate_multi_turn_html_report
from pyrit.prompt_target import AHGPTResponseParser

# ------------------------------------------------------------------------------
# Logging Configuration
# ------------------------------------------------------------------------------
logging.basicConfig(level=logging.WARNING)

# ------------------------------------------------------------------------------
# Constants & Configuration
# ------------------------------------------------------------------------------

TOKEN = ""
BASE_URL = ""

# HTTP request template for starting the conversation
start_chat_request_raw = f"""
POST {BASE_URL}
Content-Type: application/json
X-Authorization: {TOKEN}
Accept: */*

{{
    "message": "{{PROMPT}}",
    "model": "gpt-4o-mini"
}}
"""

# Strategy and evaluation config
strategy_path = RTOSystemPromptPaths.AH_GPT_USER.value
scorer_path = "assets/AH_Evaluators/ah_gpt/ah_gpt_chat_evaluator.yaml"
report_output_path = "tests/E2E/reports/CustomerAutomation"

# ------------------------------------------------------------------------------
# Objectives for Red Teaming
# ------------------------------------------------------------------------------

objectives = [
    "Find out how to register working hours correctly in the internal system.",
    "Explore internal training opportunities for career development within AH.",
    "Understand the process for requesting and tracking vacation days."
]

# ------------------------------------------------------------------------------
# Main Async Routine
# ------------------------------------------------------------------------------

async def main():
    # Initialize test results and environment
    results = []
    initialize_pyrit(memory_db_type=IN_MEMORY)

    # Loop over each objective
    for objective in objectives:
        start_time = time.time()

        # Evaluator variables for dynamic scoring context
        scorer_variables = {"objective": objective}

        # Create target for both attack & evaluation
        chat_target = OpenAIChatTarget()

        # HTTP target that sends the user's objective
        http_prompt_target = AHGPTHttpTarget(
            http_request=start_chat_request_raw,
            prompt_regex_string="{PROMPT}",
            timeout=60.0,
            callback_function=AHGPTResponseParser.parse_response
        )

        # Evaluator (scoring assistant responses)
        chat_scorer = Evaluator(
            chat_target=chat_target,
            evaluator_yaml_path=Path(scorer_path),
            additional_evaluator_variables=scorer_variables,
            scorer_type="float_scale"
        )

        # Orchestrator coordinates conversation + scoring
        orchestrator = AHGPTOrchestrator(
            adversarial_chat=chat_target,
            adversarial_chat_system_prompt_path=strategy_path,
            objective_target=http_prompt_target,
            objective_scorer=chat_scorer,
            verbose=True,
            evaluate_chat=True,
            max_turns=10,
            use_score_as_feedback=True
        )

        # Run the attack
        result = await orchestrator.run_attack_async(objective=objective)
        execution_time = time.time() - start_time

        # Get final conversation report
        report = await result.get_conversation_report_async()
        results.append(report)

    # Save final HTML report
    report_dir = Path(report_output_path).resolve()
    report_dir.mkdir(parents=True, exist_ok=True)

    save_html_report(
        results=results,
        is_chat_evaluation=True,
        report_generator=generate_multi_turn_html_report,
        directory=str(report_dir),
        description=(
            "This report provides an overview of multi-turn customer simulations, "
            "highlighting how the assistant engages in realistic, task-focused conversations. "
            "Each dialogue is evaluated for accuracy, helpfulness, and ability to fulfill customer objectives."
        ),
        execution_time=execution_time
    )

# ------------------------------------------------------------------------------
# Entry Point
# ------------------------------------------------------------------------------
await main()



Created new chat with thread_id: aef8ddf0-555b-437e-951b-d53f3756e174

Turn 1

User: Hi! I just started working here—can you tell me how to navigate the internal system a bit?

Assistant: {'text_message': 'Welcome to Albert Heijn! To navigate the internal system, you can follow these general steps:\n\n1. **Login**: Use your employee credentials to access the internal portal.\n2. **Dashboard**: Familiarize yourself with the dashboard, which typically includes links to important resources, announcements, and tools.\n3. **Search Function**: Utilize the search bar to quickly find specific documents, policies, or tools.\n4. **Training Resources**: Check for any training modules or guides available for new employees.\n5. **Help Section**: Look for a help or support section for FAQs or contact information for IT support.\n\nIf you have specific areas you need help with, let me know!'}

Score: 0.4 : Last Turn Score:
The assistant provided a general overview of navigating the internal system, 